In [ ]:
%pylab

<div id='CheckNiDecay'/>
### Energy release in Ni56->Co56->Fe56 decay prefactors

In [ ]:
day = 86400;
MeV=1.6022e-06; #erg
atomicmassunit=1.660538921e-24; #gram
DmNi56=-53900;  # mass excess from Audi 95, Nuc. Phys. A. 595, 409
mNi56=(56+DmNi56/931493.86)*atomicmassunit;
Msun= 1.9889e+33;

Parameters from [Nadyozhin 1994](http://adsabs.harvard.edu/abs/1994ApJS...92..527N) based on Huo et. al.  1987 Nuclear data sheets 51, 1:

In [ ]:
tauNi=8.8*day             #56Ni decay time
tauCo=111.3*day;          #56Co decay time
QNi=1.75*MeV;             #56Ni released energy per decay, ergs
QCo_ga=3.61*MeV;          #56Co released energy in gammas, including 0.511 MeV line from positron anihilation
QCo_pos=0.12*MeV;         #kinetic energy in positrons

Alternatively use parameters from other sources (run either the cell above or the following):

In [ ]:
tauNi=8.7644*day             #56Ni decay time 
tauCo=111.42*day;          #56Co decay time
                        # from Junde 1999
QNi=1.728*MeV;             #56Ni released energy per decay, ergs
QCo_ga=3.5658*MeV;          #56Co released energy in gammas, including 0.511 MeV line from positron anihilation
                            # from Ambwani & Sutherland 1988
QCo_pos=0.12*MeV;         #kinetic energy in positrons

Calculate the decay function:

In [ ]:
QCo=QCo_ga+QCo_pos;       #total energy per 56Co decay

preCo_ga=QCo_ga/(tauCo-tauNi)*Msun/mNi56
preNi_ga=QNi/tauNi*Msun/mNi56 - preCo_ga

print('Qgamma=MNi/Msun*(%0.3e exp(-t/8.76d) + %0.3e exp(-t/111.42d)) ergs/s'%(preNi_ga,preCo_ga))
preCo_pos=QCo_pos/(tauCo-tauNi)*Msun/mNi56
preNi_pos=-preCo_pos;
print('Qpos=MNi/Msun*(%0.3e [-exp(-t/8.76d) + exp(-t/111.42d)] ergs/s'%preCo_pos)
print('Qdec_tot=MNi/Msun*(%0.3e exp(-t/8.8d) + %0.3e exp(-t/111.3d)) ergs/s'%(preNi_ga+preNi_pos,preCo_ga+preCo_pos))

For comparison, equation 4 in page 24 in the PhD thesis of Maximilian Stritzinger has

$$Q_{dec,tot}=\frac{M_{Ni}}{Msun}\left[6.45e^{-t/8.8d}+1.45e^{-t/111.3d}\right]\times 10^{43}\rm erg/s$$

### gamma-ray effective opacity

First we need to create functions for Compton scattering cross section and average energy loss.

In [ ]:
def Compton_E_E0(x,coste):
    return 1./(1+x*(1-coste));
def Compton_dsig_dcoste(x,coste):
    f=Compton_E_E0(x,coste);
    sin2te=1-coste**2;
    dsigma_dcoste=3/(16*pi)*f**2*(f+1/f-sin2te);
    return dsigma_dcoste

Next we need to get the branching ratio of the different gamma-ray lines.

In [ ]:
Ncoste=1000;
sigT =6.6520e-25; #Thompson Cross section 
mp =1.6730e-24;
mec2MeV=0.511;
coste_v=linspace(-1,1,Ncoste); 

EMev_I1=array([ #http://adsabs.harvard.edu/abs/1994ApJS...92..527N
[0.511,38],
[0.847,100],
[0.990,2.8],
[1.040,14],
[1.240,68],
[1.360,4.3],
[2.030,12],
[2.600,17],
[3.240,12.5],
[1.770,16],
])
EMev_I2=array([  #http://arxiv.org/abs/1308.6352
[0.511,38],
[0.847,100],
[0.977,1.4],
[1.038,14.0],
[1.175,2.3],
[1.238,67.6],
[1.360,4.3],
[1.771,15.7],
[2.015,3.1],
[2.035,7.9],
[2.598,17.3],
[3.010,1.0],
[3.202,3.2],
[3.253,7.9],
[3.273,1.9],
])

EMev_I=EMev_I1;
EMev=EMev_I[:,0];
x_v=EMev/mec2MeV;
I_v=EMev_I[:,1];
QgaMeV=sum(EMev*I_v)/100
#
sigtotE_v=I_v*0; sigtot_v=I_v*0; E_eff=0;
for l in range(len(x_v)):
    x=x_v[l];
    sigtot_v[l]=2*pi*trapz(Compton_dsig_dcoste(x,coste_v),coste_v);
    sigtotE_v[l]=2*pi*trapz(Compton_dsig_dcoste(x,coste_v)*(x-x*Compton_E_E0(x,coste_v)),coste_v);


dE_E=sigtotE_v/sigtot_v/x_v;

sigmeanE=sum(sigtotE_v*I_v)/sum(x_v*I_v)
kappa_effE_mue=sigmeanE*sigT/(2*mp)
print('kappa_eff=',kappa_effE_mue)
print('Jeffery 99 says kappa_effE_mue=kappa_c=',0.05/2)